In [1]:
import requests
import pandas as pd
import time
import re
import datetime

# ------------------ API ------------------
LISTING_URL = "https://gateway.chotot.com/v1/public/ad-listing"
DETAIL_URL = "https://gateway.chotot.com/v1/public/ad-listing/{}"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Origin": "https://www.nhatot.com",
    "Referer": "https://www.nhatot.com/",
    "cookie": "__cf_bm=PASTE_YOUR_COOKIE_HERE"  # ⚠️ Thay cookie thật khi chạy thật
}

PARAMS_BASE = {
    "region_v2": 3017,  # Đà Nẵng
    "cg": 1050,         # Cho thuê phòng trọ
    "st": "u,h",
    "limit": 50,
}

# ------------------ HÀM PHỤ ------------------
def detect_features(text):
    """Trích xuất tiện nghi từ mô tả"""
    if not text:
        return {"Máy giặt": 0, "Tủ lạnh": 0, "Wifi": 0, "Điều hòa": 0}
    t = text.lower()
    return {
        "Máy giặt": 1 if re.search(r"máy[\s\-]?giặt|may giat|giặt", t) else 0,
        "Tủ lạnh": 1 if re.search(r"tủ[\s\-]?lạnh|tu lanh|tulanh", t) else 0,
        "Wifi": 1 if re.search(r"wi[-\s]?fi|wifi", t) else 0,
        "Điều hòa": 1 if re.search(r"điều[\s\-]?hòa|dieu hoa|máy[\s\-]?lạnh|đh\b", t) else 0,
    }

def safe_get_detail(ad_id, headers, max_retries=3):
    """Lấy mô tả chi tiết tin đăng"""
    backoff = 0.5
    for _ in range(max_retries):
        try:
            r = requests.get(DETAIL_URL.format(ad_id), headers=headers, timeout=10)
            if r.status_code == 200:
                d = r.json()
                return d.get("body", "") or ""
            time.sleep(backoff)
        except Exception:
            time.sleep(backoff)
        backoff *= 1.8
    return ""

# ------------------ CÀO DỮ LIỆU ------------------
all_data = []
offset = 0
limit = PARAMS_BASE["limit"]
page = 1

while True:
    params = {**PARAMS_BASE, "o": offset}
    try:
        r = requests.get(LISTING_URL, params=params, headers=HEADERS, timeout=15)
        if r.status_code != 200:
            print(f"⚠️ Lỗi {r.status_code} tại offset={offset}")
            break
        data = r.json()
        ads = data.get("ads", [])
        total_ads = data.get("total_ads")
        print(f"📄 Trang {page} | Offset {offset} → {len(ads)} tin (Total_ads={total_ads})")
    except Exception as e:
        print(f"⚠️ Lỗi kết nối: {e}")
        break

    if not ads:
        break

    for ad in ads:
        ad_id = ad.get("list_id")
        if not ad_id:
            continue

        price_str = ad.get("price_string", "")
        if not price_str:
            continue

        # --- Ngày đăng ---
        list_time = ad.get("list_time")
        post_date = None
        if list_time and isinstance(list_time, (int, float)):
            try:
                if list_time > 1e12:
                    list_time /= 1000
                post_date = datetime.datetime.fromtimestamp(list_time).strftime("%d/%m/%Y")
            except Exception:
                post_date = None

        # --- Lấy mô tả ---
        desc = safe_get_detail(ad_id, HEADERS)
        if not desc:
            desc = ad.get("body", "") or ""

        features = detect_features(desc)

        all_data.append({
            "Location": ad.get("area_name", ""),
            "Price": price_str,
            "Area": f"{ad.get('size')} m²" if ad.get("size") else None,
            "Ngày đăng": post_date,
            **features,
        })

        time.sleep(0.3)

    offset += limit
    page += 1

    if total_ads and offset >= total_ads:
        break

    time.sleep(0.8)

# ------------------ XỬ LÝ DỮ LIỆU ------------------
df = pd.DataFrame(all_data)

def clean_price(price):
    if pd.isna(price):
        return None
    text = str(price).lower().strip()
    match = re.search(r'(\d+(\.\d+)?)', text)
    if not match:
        return None
    value = float(match.group(1))
    if "triệu" in text:
        value *= 1_000_000
    elif "nghìn" in text or "k" in text:
        value *= 1_000
    elif "tỷ" in text:
        value *= 1_000_000_000
    return value

def clean_area(area):
    if pd.isna(area):
        return None
    match = re.search(r"(\d+(\.\d+)?)", str(area))
    return float(match.group(1)) if match else None

df["Price"] = df["Price"].apply(clean_price)
df["Area"] = df["Area"].apply(clean_area)

# ------------------ IN RA KẾT QUẢ (CHO GITHUB ACTIONS) ------------------
print("\n✅ Hoàn tất cào dữ liệu!")
print(df.to_csv(index=False, encoding="utf-8-sig"))


📄 Trang 1 | Offset 0 → 50 tin (Total_ads=None)
📄 Trang 2 | Offset 50 → 50 tin (Total_ads=None)
📄 Trang 3 | Offset 100 → 50 tin (Total_ads=None)
📄 Trang 4 | Offset 150 → 31 tin (Total_ads=None)
📄 Trang 5 | Offset 200 → 0 tin (Total_ads=None)

✅ Hoàn tất cào dữ liệu!
Location,Price,Area,Ngày đăng,Máy giặt,Tủ lạnh,Wifi,Điều hòa
Quận Ngũ Hành Sơn,5000000.0,65.0,03/11/2025,0,0,0,1
Quận Ngũ Hành Sơn,3000000.0,35.0,03/11/2025,1,0,0,1
Quận Ngũ Hành Sơn,4000000.0,30.0,03/11/2025,0,0,0,0
Quận Hải Châu,2000000.0,18.0,03/11/2025,1,0,1,1
Quận Ngũ Hành Sơn,5000000.0,103.0,03/11/2025,0,0,0,0
Quận Thanh Khê,2000000.0,15.0,03/11/2025,0,1,1,1
Quận Thanh Khê,3000000.0,35.0,03/11/2025,1,1,1,1
Quận Ngũ Hành Sơn,3000000.0,30.0,03/11/2025,0,1,0,1
Quận Sơn Trà,2000000.0,30.0,03/11/2025,0,0,0,0
Quận Ngũ Hành Sơn,2000000.0,20.0,03/11/2025,0,0,0,0
Quận Ngũ Hành Sơn,2000000.0,15.0,02/11/2025,1,0,1,1
Quận Sơn Trà,4000000.0,35.0,02/11/2025,0,0,0,0
Quận Cẩm Lệ,1000000.0,15.0,02/11/2025,1,1,0,0
Quận Liên Chiểu,200000